In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random as rn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from itertools import product

market_data = pd.read_excel(r'real-estate-valuation.xlsx')

In [2]:
market_data = market_data.rename(columns={'X1 transaction date': 'transaction date',                 
                                          'X2 house age': 'house age',                            
                                          'X3 distance to the nearest MRT station': 'distance to the nearest MRT station',
                                          'X4 number of convenience stores': 'number of convenience stores',
                                          'X5 latitude': 'latitude',                
                                          'X6 longitude': 'longitude',                 
                                          'Y house price of unit area': 'house price of unit area'})

In [3]:
print(market_data.info())
print(market_data.head())
print(market_data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414 entries, 0 to 413
Data columns (total 8 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   No                                   414 non-null    int64  
 1   transaction date                     414 non-null    float64
 2   house age                            414 non-null    float64
 3   distance to the nearest MRT station  414 non-null    float64
 4   number of convenience stores         414 non-null    int64  
 5   latitude                             414 non-null    float64
 6   longitude                            414 non-null    float64
 7   house price of unit area             414 non-null    float64
dtypes: float64(6), int64(2)
memory usage: 26.0 KB
None
   No  transaction date  house age  distance to the nearest MRT station  \
0   1       2012.916667       32.0                             84.87882   
1   2       2012.916667       

In [4]:
seed = 1
tf.random.set_seed(seed)
rn.seed(seed)
np.random.seed(seed)
tf.config.experimental.enable_op_determinism()

In [5]:
X = market_data.drop(['No'], axis=1)
y = market_data['house price of unit area']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

In [6]:
neurons_list = [20, 30, 40, 50, 60]
learning_rate_list = [0.001, 0.01, 0.1]
batch_size_list = [16, 32, 64]
epochs_list = [25, 50, 75, 100]

param_grid = list(product(neurons_list, learning_rate_list, batch_size_list, epochs_list))

best_params = None
best_loss = np.inf

for neurons, learning_rate, batch_size, epochs in param_grid:
    neural_network = tf.keras.Sequential([
        tf.keras.Input(shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(neurons, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal(seed=seed)),
        tf.keras.layers.Dense(neurons, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal(seed=seed)),
        tf.keras.layers.Dense(1, kernel_initializer=tf.keras.initializers.HeNormal(seed=seed))
    ])

    neural_network.compile(
        loss='mean_squared_error',
        optimizer=tf.keras.optimizers.Adam(learning_rate)
    )
    
    neural_network.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=0)

    val_loss = neural_network.evaluate(X_test, y_test, verbose=0)

    if val_loss < best_loss:
        best_loss = val_loss
        best_params = (neurons, learning_rate, batch_size, epochs)

print(f"Best Parameters: Neurons={best_params[0]}, Learning Rate={best_params[1]}, Batch Size={best_params[2]}, Epochs={best_params[3]}")
print(f"Best Validation Loss: {best_loss}")

Best Parameters: Neurons=20, Learning Rate=0.1, Batch Size=16, Epochs=75
Best Validation Loss: 0.27947139739990234


In [7]:
neural_network = tf.keras.Sequential([
    tf.keras.Input(shape=(X_test.shape[1],)),
    tf.keras.layers.Dense(20, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal(seed=seed)),
    tf.keras.layers.Dense(20, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal(seed=seed)),
    tf.keras.layers.Dense(1, kernel_initializer=tf.keras.initializers.HeNormal(seed=seed))
])

neural_network.compile(
    loss='mean_squared_error',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1)
)

history = neural_network.fit(X_train, y_train, batch_size=16, epochs=75, verbose=0)

In [10]:
neural_network.evaluate(X_test, y_test, verbose=0)

0.27947139739990234